In [1]:
import os
import tempfile
import warnings
from os.path import join as pjoin

import dask
import dask.dataframe as dd
import dask_geopandas as dgpd
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from scipy.fft import dst
import tqdm
import xarray as xr
from dask.diagnostics import ProgressBar
from rasterio.crs import CRS

from raster_tools import Raster, Vector, open_vectors, clipping, zonal
from raster_tools.dtypes import F32, U8

import matplotlib.pyplot as plt

In [8]:
# change pandas max col display
pd.set_option('display.max_columns', 500)

In [2]:
# Location for temporary storage
TMP_LOC = "/home/jake/FireLab/Project/data/temp/"
DATA_LOC = "/home/jake/FireLab/Project/data/"

STATE = "OR"

# Location of clipped DEM files
DEM_DATA_DIR = pjoin(TMP_LOC, "dem_data")

# location of feature data files
FEATURE_DIR = pjoin(DATA_LOC, "FeatureData")
EDNA_DIR = pjoin(DATA_LOC, "terrain")
MTBS_DIR = pjoin(DATA_LOC, "MTBS_Data")

mtbs_df_path = pjoin(TMP_LOC, f"{STATE}_mtbs.parquet")
mtbs_df_temp_path = pjoin(TMP_LOC, f"{STATE}_mtbs_temp.parquet")
checkpoint_1_path = pjoin(TMP_LOC, "check1")
checkpoint_2_path = pjoin(TMP_LOC, "check2")

PATHS = {
    "states": pjoin(EDNA_DIR, "state_borders/cb_2018_us_state_5m.shp"),
    "dem": pjoin(EDNA_DIR, "us_orig_dem/us_orig_dem/orig_dem/hdr.adf"),
    "dem_slope": pjoin(EDNA_DIR, "us_slope/us_slope/slope/hdr.adf"),
    "dem_aspect": pjoin(EDNA_DIR, "us_aspect/aspect/hdr.adf"),
    "dem_flow_acc": pjoin(EDNA_DIR, "us_flow_acc/us_flow_acc/flow_acc/hdr.adf"),
    "gm_srad": pjoin(FEATURE_DIR, "gridmet/srad_1986_2020_weekly.nc"),
    "gm_vpd": pjoin(FEATURE_DIR, "gridmet/vpd_1986_2020_weekly.nc"),
    "aw_mat": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MAT.tif"),
    "aw_mcmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MCMT.tif"),
    "aw_mwmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MWMT.tif"),
    "aw_td": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_TD.tif"),
    "dm_tmax": pjoin(FEATURE_DIR, "daymet/tmax_1986_2020.nc"),
    "dm_tmin": pjoin(FEATURE_DIR, "daymet/tmin_1986_2020.nc"),
    "biomass_afg": pjoin(
        FEATURE_DIR, "biomass/biomass_afg_1986_2020_{}.nc".format(STATE)
    ),
    "biomass_pfg": pjoin(
        FEATURE_DIR, "biomass/biomass_pfg_1986_2020_{}.nc".format(STATE)
    ),
    "landfire_fvt": pjoin(
        FEATURE_DIR, "landfire/LF2020_FVT_200_CONUS/Tif/LC20_FVT_200.tif"
    ),
    "landfire_fbfm40": pjoin(
        FEATURE_DIR, "landfire/LF2020_FBFM40_200_CONUS/Tif/LC20_F40_200.tif"
    ),
    "ndvi": pjoin(FEATURE_DIR, "ndvi/access/weekly/ndvi_1986_2020_weekavg.nc"),
    "mtbs_root": pjoin(MTBS_DIR, "MTBS_BSmosaics/"),
    "mtbs_perim": pjoin(MTBS_DIR, "mtbs_perimeter_data/mtbs_perims_DD.shp"),
}
YEARS = list(range(1986, 2021))
GM_KEYS = list(filter(lambda x: x.startswith("gm_"), PATHS))
AW_KEYS = list(filter(lambda x: x.startswith("aw_"), PATHS))
DM_KEYS = list(filter(lambda x: x.startswith("dm_"), PATHS))
BIOMASS_KEYS = list(filter(lambda x: x.startswith("biomass_"), PATHS))
LANDFIRE_KEYS = list(filter(lambda x: x.startswith("landfire_"), PATHS))
NDVI_KEYS = list(filter(lambda x: x.startswith("ndvi"), PATHS))
DEM_KEYS = list(filter(lambda x: x.startswith("dem"), PATHS))

In [ ]:
with ProgressBar():
    df = dgpd.read_parquet(checkpoint_2_path)

In [ ]:
nc_ds = xr.open_dataset(PATHS["aw_mat"], chunks={"day": 1})

In [ ]:
nc_ds.variables

In [ ]:
# a function for comparing and displaying data / coords / info from a collection of xarray datasets
def compare_xr_datasets(datasets, attrs=False, coords=False, info=False):
    for ds in datasets:
        print(f"{ds = }")
        if attrs:
            print(f"{ds.attrs = }")
        if coords:
            print(f"{ds.coords = }")
        if info:
            print(f"{ds.info = }")
        print("-----------------\n\n")

nc_datasets = [xr.open_dataset(PATHS['dm_tmax'], chunks={"day": 1}), xr.open_dataset(PATHS['aw_mat'], chunks={"day": 1})]
compare_xr_datasets(nc_datasets, attrs=True, coords=True, info=True)

In [ ]:
nc_ds = xr.open_dataset(PATHS['dm_tmax'], chunks={"day": 1})#, decode_times=False)
nc_ds = nc_ds.rio.write_crs("EPSG:5071")  # FOR DAYMET ONLY!!
nc_ds = nc_ds.rio.write_crs(
    nc_ds.coords["lambert_conformal_conic"].spatial_ref
)  # FOR DAYMET ONLY!!
nc_ds = nc_ds.rename({"lambert_conformal_conic": "crs"})  # FOR DAYMET ONLY!!
nc_ds2 = nc_ds.drop_vars(["lat", "lon"])  # FOR DAYMET ONLY!!
nc_ds2 = nc_ds2.rename_vars({"x": "lon", "y": "lat"})  # FOR DAYMET ONLY!!
nc_ds2

In [ ]:
# visually map/draw the fire from df with ig_date = 1986-03-20
import matplotlib.pyplot as plt
# plot with oregon map as background
fig, ax = plt.subplots(figsize=(15,15))
states = gpd.read_file(PATHS['states'])
states = states.to_crs(df.crs)
# limit states to just oregon
states = states[states.STUSPS == 'OR']
states.plot(ax=ax, color='white', edgecolor='black')
df[df.ig_date == '2014-07-14'].plot(ax=ax, color='red')
plt.show()

---

## MTBS DATASET INFO

In [3]:
df = dgpd.read_parquet(mtbs_df_temp_path)
df.head()

,mtbs,geometry,state,ig_date,dem,dem_slope,dem_aspect,dem_flow_acc,dm_tmax,dm_tmin,...,gm_vpd,ndvi,aw_mat,aw_mcmt,aw_mwmt,aw_td,landfire_fvt,landfire_fbfm40,hillshade,year
0,2,POINT (-1830990.000 2456610.000),OR,1986-03-20,1261.0,0.559452,305.295502,0.0,-3.049677,-16.621613,...,0.54,0.1412,8.7,-1.9,20.9,22.799999,2967.0,102.0,76,1986
1,2,POINT (-1830960.000 2456610.000),OR,1986-03-20,1277.0,2.485488,128.388641,0.0,-3.049677,-16.621613,...,0.54,0.1412,8.7,-1.9,20.9,22.799999,2967.0,102.0,187,1986
2,2,POINT (-1830990.000 2456580.000),OR,1986-03-20,1409.0,10.548834,277.054626,6.0,-3.049677,-16.621613,...,0.54,0.1412,8.7,-1.9,20.9,22.799999,2967.0,102.0,133,1986
3,2,POINT (-1830960.000 2456580.000),OR,1986-03-20,1545.0,7.670347,30.625286,0.0,-3.049677,-16.621613,...,0.54,0.1412,8.7,-1.9,20.9,22.799999,2080.0,122.0,63,1986
4,2,POINT (-1830930.000 2456580.000),OR,1986-03-20,1407.0,1.268448,124.799530,43.0,-3.049677,-16.621613,...,0.54,0.1412,8.7,-1.9,20.9,22.799999,2123.0,122.0,181,1986


In [20]:
mtbs_dataset = df.compute()

In [8]:
mtbs_dataset.shape

(55328732, 23)

In [21]:
# a function to round numeric values in a dataframe to 3 decimals (if they are > 1)
def round_df(df):
    columns_to_round = ['aw_mwmt', 'dm_tmax', 'dm_tmin', 'gm_srad', 'gm_vpd', 'aw_td', 'aw_mcmt', 'dem_aspect', 'dem_slope']
    for col in columns_to_round:
        df[col] = df[col].round(3)
    return df

In [9]:
mtbs_dataset

,mtbs,geometry,state,ig_date,dem,dem_slope,dem_aspect,dem_flow_acc,dm_tmax,dm_tmin,biomass_afg,biomass_pfg,gm_srad,gm_vpd,ndvi,aw_mat,aw_mcmt,aw_mwmt,aw_td,landfire_fvt,landfire_fbfm40,hillshade,year
0,2,POINT (-1830990.000 2456610.000),OR,1986-03-20,1261.0,0.559452,305.295502,0.0,-3.049677,-16.621613,165.0,891.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2967.0,102.0,76.0,1986
1,2,POINT (-1830960.000 2456610.000),OR,1986-03-20,1277.0,2.485488,128.388641,0.0,-3.049677,-16.621613,411.0,638.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2967.0,102.0,187.0,1986
2,2,POINT (-1830990.000 2456580.000),OR,1986-03-20,1409.0,10.548834,277.054626,6.0,-3.049677,-16.621613,388.0,508.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2967.0,102.0,133.0,1986
3,2,POINT (-1830960.000 2456580.000),OR,1986-03-20,1545.0,7.670347,30.625286,0.0,-3.049677,-16.621613,411.0,638.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2080.0,122.0,63.0,1986
4,2,POINT (-1830930.000 2456580.000),OR,1986-03-20,1407.0,1.268448,124.799530,43.0,-3.049677,-16.621613,379.0,492.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2123.0,122.0,181.0,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425327,1,POINT (-1697400.000 2447790.000),OR,2020-11-05,1528.0,15.068486,83.439735,47.0,-12.428667,-21.412666,579.0,558.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2273.0,102.0,141.0,2020
425328,1,POINT (-1697370.000 2447790.000),OR,2020-11-05,1521.0,14.851972,84.313797,48.0,-12.428667,-21.412666,710.0,332.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2273.0,102.0,142.0,2020
425329,1,POINT (-1697490.000 2447760.000),OR,2020-11-05,1513.0,14.268291,85.838928,49.0,-12.428667,-21.412666,753.0,400.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2134.0,101.0,143.0,2020
425330,1,POINT (-1697460.000 2447760.000),OR,2020-11-05,1506.0,12.509232,89.095985,50.0,-12.428667,-21.412666,701.0,481.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2134.0,101.0,146.0,2020


In [18]:
mtbs_dataset.dtypes

mtbs                        uint8
geometry                 geometry
state                      object
ig_date            datetime64[ns]
dem                       float32
dem_slope                 float32
dem_aspect                float32
dem_flow_acc              float32
dm_tmax                   float32
dm_tmin                   float32
biomass_afg               float32
biomass_pfg               float32
gm_srad                   float32
gm_vpd                    float32
ndvi                      float32
aw_mat                    float32
aw_mcmt                   float32
aw_mwmt                   float32
aw_td                     float32
landfire_fvt              float32
landfire_fbfm40           float32
hillshade                 float64
year                       uint16
dtype: object

In [17]:
mtbs_dataset = round_df(mtbs_dataset)
mtbs_dataset

,mtbs,geometry,state,ig_date,dem,dem_slope,dem_aspect,dem_flow_acc,dm_tmax,dm_tmin,biomass_afg,biomass_pfg,gm_srad,gm_vpd,ndvi,aw_mat,aw_mcmt,aw_mwmt,aw_td,landfire_fvt,landfire_fbfm40,hillshade,year
0,2,POINT (-1830990.000 2456610.000),OR,1986-03-20,1261.0,0.559,305.295990,0.0,-3.050,-16.622,165.0,891.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2967.0,102.0,76.0,1986
1,2,POINT (-1830960.000 2456610.000),OR,1986-03-20,1277.0,2.485,128.389008,0.0,-3.050,-16.622,411.0,638.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2967.0,102.0,187.0,1986
2,2,POINT (-1830990.000 2456580.000),OR,1986-03-20,1409.0,10.549,277.054993,6.0,-3.050,-16.622,388.0,508.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2967.0,102.0,133.0,1986
3,2,POINT (-1830960.000 2456580.000),OR,1986-03-20,1545.0,7.670,30.625000,0.0,-3.050,-16.622,411.0,638.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2080.0,122.0,63.0,1986
4,2,POINT (-1830930.000 2456580.000),OR,1986-03-20,1407.0,1.268,124.800003,43.0,-3.050,-16.622,379.0,492.0,174.600006,0.54,0.1412,8.7,-1.9,20.900000,22.799999,2123.0,122.0,181.0,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425327,1,POINT (-1697400.000 2447790.000),OR,2020-11-05,1528.0,15.068,83.440002,47.0,-12.429,-21.413,579.0,558.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2273.0,102.0,141.0,2020
425328,1,POINT (-1697370.000 2447790.000),OR,2020-11-05,1521.0,14.852,84.314003,48.0,-12.429,-21.413,710.0,332.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2273.0,102.0,142.0,2020
425329,1,POINT (-1697490.000 2447760.000),OR,2020-11-05,1513.0,14.268,85.838997,49.0,-12.429,-21.413,753.0,400.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2134.0,101.0,143.0,2020
425330,1,POINT (-1697460.000 2447760.000),OR,2020-11-05,1506.0,12.509,89.096001,50.0,-12.429,-21.413,701.0,481.0,83.800003,0.56,0.0792,9.0,-1.6,21.799999,23.400000,2134.0,101.0,146.0,2020


In [ ]:
ndvi_ds = xr.open_dataset(PATHS['ndvi'], chunks={"day": 1})

In [ ]:
ndvi_ds.crs

In [3]:
# load mtbs perimeters
print("Loading MTBS perimeters")
mtbs_perim = gpd.read_file(PATHS["mtbs_perim"])
mtbs_perim.columns

Loading MTBS perimeters


Index(['Event_ID', 'irwinID', 'Incid_Name', 'Incid_Type', 'Map_ID', 'Map_Prog',
       'Asmnt_Type', 'BurnBndAc', 'BurnBndLat', 'BurnBndLon', 'Ig_Date',
       'Pre_ID', 'Post_ID', 'Perim_ID', 'dNBR_offst', 'dNBR_stdDv', 'NoData_T',
       'IncGreen_T', 'Low_T', 'Mod_T', 'High_T', 'Comment', 'geometry'],
      dtype='object')

In [ ]:
# extract only the columns we need (Event_ID where startswith OR, Ig_Date, and geometry)
mtbs_perim = mtbs_perim[["Event_ID", "Ig_Date", "geometry"]]
mtbs_perim = mtbs_perim[mtbs_perim.Event_ID.str.startswith("OR")]
mtbs_perim.reset_index(drop=True, inplace=True)
mtbs_perim